In [23]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random

In [3]:
img_dir = "data/"

In [4]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [17]:
#resize the image
def resize_img(img_array):
    new_array=cv2.resize(img_array, (224,224))
    # plt.imshow(new_array)
    # plt.show()
    return new_array


In [20]:
training_data=[]
def create_training_data():
    for cls in classes:
        path=os.path.join(img_dir, cls)
        class_num=classes.index(cls)
        for img in os.listdir(path):
            try:
                img_array= cv2.imread(os.path.join(path, img))
                new_array=resize_img(img_array)
                training_data.append([new_array,class_num])
            except Exception as e:
                pass   

In [21]:
create_training_data()

In [22]:
len(training_data)

1835

In [24]:
random.shuffle(training_data)

In [70]:
training_data

[[array([[[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         ...,
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255

In [25]:
X=[]
y=[]

img_size=244
for feature, label in training_data:
    X.append(feature)
    y.append(label)

X=np.array(X) 

In [26]:
X.shape

(1835, 224, 224, 3)

In [27]:
X=X/255.0  

In [29]:
Y=np.array(y)
Y

array([21,  0,  6, ...,  8,  3,  9])

## **Model Building**

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [31]:
model=tf.keras.applications.MobileNetV2()
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [32]:
base_input=model.layers[0].input
base_output=model.layers[-2].output

In [33]:
final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(30, activation='softmax')(final_output) # 30 is for 30 different classes

In [34]:
final_output

<KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'dense_2')>

In [35]:
new_model=keras.Model(inputs=base_input, outputs=final_output)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [40]:
new_model.compile(loss = 'sparse_categorical_crossentropy',  optimizer = 'adam', metrics = ["accuracy"])

## **Model Training**

In [41]:
new_model.fit(X,Y, epochs = 10)

Epoch 1/10
58/58 [==============================] - 10s 136ms/step - loss: 0.1693 - accuracy: 0.9580
Epoch 2/10
58/58 [==============================] - 7s 125ms/step - loss: 0.1405 - accuracy: 0.9646
Epoch 3/10
58/58 [==============================] - 7s 125ms/step - loss: 0.1133 - accuracy: 0.9700
Epoch 4/10
58/58 [==============================] - 7s 124ms/step - loss: 0.0773 - accuracy: 0.9782
Epoch 5/10
58/58 [==============================] - 7s 125ms/step - loss: 0.0622 - accuracy: 0.9837
Epoch 6/10
58/58 [==============================] - 7s 126ms/step - loss: 0.0639 - accuracy: 0.9837
Epoch 7/10
58/58 [==============================] - 7s 125ms/step - loss: 0.0864 - accuracy: 0.9777
Epoch 8/10
58/58 [==============================] - 7s 125ms/step - loss: 0.0412 - accuracy: 0.9880
Epoch 9/10
58/58 [==============================] - 7s 120ms/step - loss: 0.0503 - accuracy: 0.9896
Epoch 10/10
58/58 [==============================] - 7s 122ms/step - loss: 0.0273 - accuracy: 0.993

In [68]:
test_img=cv2.imread('data/29/TF-S-001.jpg')

In [69]:
final_image=cv2.resize(test_img, (224,224))
final_image=np.expand_dims(final_image, axis=0)
final_image=final_image/255.0

result=new_model.predict(final_image)

np.argmax(result)

1/1 [==============================] - 0s 19ms/step


14